In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("../Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis = "columns", how = "all")
# Drop the null rows
df = df.dropna()
df.head()

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis = 1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
y = df["koi_disposition"]

In [ ]:
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100, stratify = y)
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model using LogisticRegression

In [ ]:
classifier = LogisticRegression(solver = "newton-cg", multi_class = "auto")
classifier.fit(X_train_scaled, y_train)

print(f"LogisticRegression training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"LogisticRegression testing Data Score: {classifier.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model
param_grid = {"C": [0.1, 1, 10],
              "max_iter": [1000, 5000, 10000]}
grid = GridSearchCV(classifier, param_grid, error_score = "raise", verbose = 3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

In [ ]:
print(f"Best grid params: {grid.best_params_}")
print(f"Best grid score: {grid.best_score_}")

In [ ]:
# Make predictions with the hypertuned model
predictions = classifier.predict(X_test_scaled)
classifications = y_test.unique().tolist()

prediction_actual = {"Actual": y_test,
                     "Prediction": predictions}

prediction_df = pd.DataFrame(prediction_actual)
prediction_df = prediction_df.set_index("Actual").reset_index()
prediction_df

In [ ]:
target_names = ["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]
print(classification_report(y_test, predictions, target_names = target_names))

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
joblib.dump(classifier, "../Models/Logistic_Regression_Classifier.sav")